Abiy

In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

%matplotlib inline

In [11]:
bond_df = pd.read_json('bond_data.json')
bond_df

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('VMBS', 'open')","('VMBS', 'high')","('VMBS', 'low')","('VMBS', 'close')","('VMBS', 'volume')","('VOOG', 'open')","('VOOG', 'high')","('VOOG', 'low')","('VOOG', 'close')","('VOOG', 'volume')"
2017-10-17 04:00:00,25.23,25.2600,25.21,25.25,675884,52.590,52.611,52.5800,52.599,621174,...,52.84,52.8499,52.8000,52.820,441153,131.27,131.5900,131.2600,131.5800,29113
2017-10-18 04:00:00,25.20,25.2100,25.17,25.20,507468,52.575,52.615,52.5690,52.615,446712,...,52.79,52.7900,52.7425,52.760,389960,131.88,131.8800,131.5301,131.6299,39530
2017-10-19 04:00:00,25.25,25.2650,25.22,25.23,689206,52.635,52.645,52.6050,52.630,529644,...,52.77,52.8200,52.7300,52.730,424189,131.10,131.5000,130.8000,131.5000,32793
2017-10-20 04:00:00,25.13,25.1700,25.13,25.15,651942,52.605,52.614,52.5950,52.605,545736,...,52.65,52.6892,52.6301,52.640,286452,132.00,132.0800,131.7100,131.9500,32071
2017-10-23 04:00:00,25.15,25.1800,25.15,25.17,472528,52.625,52.640,52.6010,52.635,856312,...,52.68,52.7200,52.6700,52.720,262840,132.30,132.3000,131.2500,131.3300,29681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,26.48,26.5200,26.47,26.52,4320797,54.620,54.640,54.6100,54.640,2304580,...,53.23,53.3100,53.1950,53.310,1291571,269.93,270.8400,266.5200,266.5200,161793
2021-10-01 04:00:00,26.54,26.5800,26.51,26.58,4107214,54.610,54.660,54.6041,54.650,2054957,...,53.28,53.3400,53.2300,53.330,1174288,267.62,270.0700,264.7936,269.3000,122595
2021-10-04 04:00:00,26.53,26.5799,26.50,26.55,2181656,54.610,54.630,54.6000,54.625,1516627,...,53.36,53.3700,53.3301,53.345,683339,268.26,268.2600,262.1500,263.9300,140997
2021-10-05 04:00:00,26.52,26.5300,26.46,26.48,4690593,54.600,54.600,54.5600,54.580,1570312,...,53.33,53.3350,53.2400,53.250,896557,264.99,268.9693,264.8400,267.4250,61595


In [3]:
stock_df = pd.read_json('stock_data.json')
stock_df

,"('AAPL', 'open')","('AAPL', 'high')","('AAPL', 'low')","('AAPL', 'close')","('AAPL', 'volume')","('AMZN', 'open')","('AMZN', 'high')","('AMZN', 'low')","('AMZN', 'close')","('AMZN', 'volume')",...,"('CCOI', 'open')","('CCOI', 'high')","('CCOI', 'low')","('CCOI', 'close')","('CCOI', 'volume')","('PG', 'open')","('PG', 'high')","('PG', 'low')","('PG', 'close')","('PG', 'volume')"
2017-10-17 04:00:00,159.75,160.870,159.2300,160.47,16059637,1005.59,1011.4700,1004.380,1008.99,1487100,...,49.65,50.00,49.150,49.90,151845,93.04,93.09,92.400,92.80,4160781
2017-10-18 04:00:00,160.42,160.710,159.6000,159.77,13668418,1009.27,1022.3100,996.550,996.99,1714126,...,50.05,50.20,49.600,49.70,202954,92.78,92.95,92.010,92.78,5978281
2017-10-19 04:00:00,156.75,157.080,155.0200,155.98,32669974,990.33,991.0500,980.240,986.60,1963364,...,49.65,50.05,47.800,48.41,128608,91.70,91.99,91.140,91.63,6497132
2017-10-20 04:00:00,156.61,157.750,155.9600,156.25,18427186,993.53,994.6200,982.000,982.91,1406057,...,48.89,50.50,47.880,49.65,204367,89.72,89.96,87.590,88.22,16520870
2017-10-23 04:00:00,156.89,157.690,155.5000,156.20,16418890,986.73,986.7750,962.500,966.34,2341953,...,49.85,50.20,49.025,49.30,142925,88.63,88.86,86.870,87.29,10177891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,143.66,144.378,141.2800,141.32,76345831,3315.25,3327.8500,3273.990,3286.61,2349154,...,71.50,71.79,70.490,70.75,117708,142.64,142.74,139.770,139.78,4447637
2021-10-01 04:00:00,141.83,142.920,139.1101,142.66,81585491,3289.01,3309.1746,3255.940,3283.17,2513022,...,71.21,72.36,69.820,72.16,150384,139.93,140.23,138.320,139.62,4464732
2021-10-04 04:00:00,141.71,142.210,138.2700,139.19,86291346,3278.75,3279.9900,3176.250,3191.70,4034895,...,72.26,73.07,71.650,72.98,112374,139.39,140.40,137.605,138.80,4681942
2021-10-05 04:00:00,139.49,142.240,139.3600,141.10,68998178,3204.12,3260.7300,3202.464,3220.99,2672848,...,72.94,72.94,71.950,72.41,97653,138.87,140.52,138.720,139.35,5417250


In [15]:
crypto_df = pd.read_json('crypto_data.json')
crypto_df

,"('BTC', 'volume')","('BTC', 'price')","('BTC', 'market_cap')","('BTC', 'total_volume')","('ETH', 'volume')","('ETH', 'price')","('ETH', 'market_cap')","('ETH', 'total_volume')","('LTE', 'volume')","('LTE', 'price')","('LTE', 'market_cap')","('LTE', 'total_volume')","('LINK', 'volume')","('LINK', 'price')","('LINK', 'market_cap')","('LINK', 'total_volume')","('ADA', 'volume')","('ADA', 'price')","('ADA', 'market_cap')","('ADA', 'total_volume')"
0,1367107200000,135.300000,1.500518e+09,0.000000e+00,1438905600000,2.831620,0.000000e+00,9.062200e+04,1367107200000,4.299830,7.377339e+07,0.000000e+00,1510185600000,0.225377,7.888183e+07,3.190026e+06,1508284800000,0.026845,6.960214e+08,2.351678e+06
1,1367193600000,141.960000,1.575032e+09,0.000000e+00,1438992000000,1.330750,8.033948e+07,3.680700e+05,1367193600000,4.359400,7.493691e+07,0.000000e+00,1510272000000,0.189036,6.616275e+07,2.121175e+06,1508371200000,0.026830,6.956220e+08,2.815156e+06
2,1367280000000,135.300000,1.501657e+09,0.000000e+00,1439164800000,0.687586,4.155631e+07,4.004641e+05,1367280000000,4.182950,7.203764e+07,0.000000e+00,1510358400000,0.181223,6.342793e+07,2.229298e+06,1508457600000,0.030300,7.855800e+08,8.883473e+06
3,1367366400000,117.000000,1.298952e+09,0.000000e+00,1439251200000,1.067379,6.453901e+07,1.518998e+06,1367366400000,3.649140,6.295799e+07,0.000000e+00,1510444800000,0.170013,5.950462e+07,5.361146e+06,1508544000000,0.028588,7.412021e+08,5.308857e+06
4,1367452800000,103.430000,1.148668e+09,0.000000e+00,1439337600000,1.256613,7.601326e+07,2.073893e+06,1367452800000,3.388790,5.856534e+07,0.000000e+00,1510531200000,0.180691,6.324186e+07,2.942234e+06,1508630400000,0.027796,7.206698e+08,2.901876e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1490227200000,1029.139414,1.670585e+10,8.204883e+07,1561939200000,292.838876,3.117638e+10,1.445223e+10,1490227200000,3.979628,2.002527e+08,1.672097e+06,1633132800000,26.310431,1.198544e+10,1.209727e+09,1631232000000,2.518800,8.086493e+10,5.008905e+09
1424,1490313600000,940.020627,1.526129e+10,1.618019e+08,1562025600000,295.255898,3.149897e+10,1.454150e+10,1490313600000,4.141265,2.084447e+08,7.600280e+06,1633219200000,26.974248,1.231979e+10,9.750935e+08,1631318400000,2.377017,7.601103e+10,5.063772e+09
1425,1490400000000,965.265063,1.567277e+10,1.904511e+08,1562112000000,290.959672,3.095935e+10,1.363409e+10,1490400000000,4.086412,2.057418e+08,5.199257e+06,1633305600000,27.609012,1.262701e+10,9.148422e+08,1631404800000,2.613539,8.364405e+10,7.277175e+09
1426,1490486400000,954.476660,1.549953e+10,1.304905e+08,1562198400000,304.170542,3.246348e+10,1.385265e+10,1490486400000,4.024873,2.027008e+08,2.335289e+06,1633392000000,26.605281,1.218513e+10,1.010408e+09,1631491200000,2.597916,8.356769e+10,7.402829e+09


In [5]:
#zero_risk=bond_df[["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ"]]

In [39]:
new_df = pd.concat([bond_df,stock_df], axis=1)
new_df

ticker_names =["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
ticker_columns = []
for column in columns:
    for ticker_name in ticker_names:
        if ticker_name in column:
            ticker_columns.append(column)
    

ticker_columns
new_df[ticker_columns]

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('VMBS', 'open')","('VMBS', 'high')","('VMBS', 'low')","('VMBS', 'close')","('VMBS', 'volume')","('VOOG', 'open')","('VOOG', 'high')","('VOOG', 'low')","('VOOG', 'close')","('VOOG', 'volume')"
2017-10-17 04:00:00,25.23,25.2600,25.21,25.25,675884.0,52.590,52.611,52.5800,52.599,621174.0,...,52.84,52.8499,52.8000,52.820,441153.0,131.27,131.5900,131.2600,131.5800,29113.0
2017-10-18 04:00:00,25.20,25.2100,25.17,25.20,507468.0,52.575,52.615,52.5690,52.615,446712.0,...,52.79,52.7900,52.7425,52.760,389960.0,131.88,131.8800,131.5301,131.6299,39530.0
2017-10-19 04:00:00,25.25,25.2650,25.22,25.23,689206.0,52.635,52.645,52.6050,52.630,529644.0,...,52.77,52.8200,52.7300,52.730,424189.0,131.10,131.5000,130.8000,131.5000,32793.0
2017-10-20 04:00:00,25.13,25.1700,25.13,25.15,651942.0,52.605,52.614,52.5950,52.605,545736.0,...,52.65,52.6892,52.6301,52.640,286452.0,132.00,132.0800,131.7100,131.9500,32071.0
2017-10-23 04:00:00,25.15,25.1800,25.15,25.17,472528.0,52.625,52.640,52.6010,52.635,856312.0,...,52.68,52.7200,52.6700,52.720,262840.0,132.30,132.3000,131.2500,131.3300,29681.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,26.48,26.5200,26.47,26.52,4320797.0,54.620,54.640,54.6100,54.640,2304580.0,...,53.23,53.3100,53.1950,53.310,1291571.0,269.93,270.8400,266.5200,266.5200,161793.0
2021-10-01 04:00:00,26.54,26.5800,26.51,26.58,4107214.0,54.610,54.660,54.6041,54.650,2054957.0,...,53.28,53.3400,53.2300,53.330,1174288.0,267.62,270.0700,264.7936,269.3000,122595.0
2021-10-04 04:00:00,26.53,26.5799,26.50,26.55,2181656.0,54.610,54.630,54.6000,54.625,1516627.0,...,53.36,53.3700,53.3301,53.345,683339.0,268.26,268.2600,262.1500,263.9300,140997.0
2021-10-05 04:00:00,26.52,26.5300,26.46,26.48,4690593.0,54.600,54.600,54.5600,54.580,1570312.0,...,53.33,53.3350,53.2400,53.250,896557.0,264.99,268.9693,264.8400,267.4250,61595.0


In [ ]:
# to_merge_dict = (bond_df,stock_df,crypto_df)
# to_merge_dict

In [ ]:
# to_merge_dict = {
#     'Crypto': crypto_df , 'Stock': stock_df,
#     'Bond': bond_df}
# merged_df = pd.concat(to_merge_dict.values(), keys=to_merge_dict.keys(), axis=1, join='inner')
# merged_df

In [ ]:
import yfinance as yf 
data = yf.download(["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ","AAPL","AMZN","CCOI", "PG","BA","NVDA","BRK-A","GS","COP","GILD","BTC","ETH","LTE.V","ADA-USD" ],'2017-01-01','2021-10-04')
data.dropna()

In [ ]:
#zero_risk=data["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ"]
data.columns = data.columns.get_level_values(0)
data.columns

In [ ]:
#!pip install pandas_datareader ,"IWF","IGSB","GOVT","AGZ"

In [ ]:
#Based on Risk level
#Investments That Provide Various Levels Of Risk & Return

In [40]:
#Zero risk portfolio: Treasury bonds held to maturity, money market accounts, and CDs where the FDIC guarantees up to $250,000 in losses per person.
#  Treasury bonds(100% )
zero_risk=new_df[ticker_columns]
zero_risk

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('VMBS', 'open')","('VMBS', 'high')","('VMBS', 'low')","('VMBS', 'close')","('VMBS', 'volume')","('VOOG', 'open')","('VOOG', 'high')","('VOOG', 'low')","('VOOG', 'close')","('VOOG', 'volume')"
2017-10-17 04:00:00,25.23,25.2600,25.21,25.25,675884.0,52.590,52.611,52.5800,52.599,621174.0,...,52.84,52.8499,52.8000,52.820,441153.0,131.27,131.5900,131.2600,131.5800,29113.0
2017-10-18 04:00:00,25.20,25.2100,25.17,25.20,507468.0,52.575,52.615,52.5690,52.615,446712.0,...,52.79,52.7900,52.7425,52.760,389960.0,131.88,131.8800,131.5301,131.6299,39530.0
2017-10-19 04:00:00,25.25,25.2650,25.22,25.23,689206.0,52.635,52.645,52.6050,52.630,529644.0,...,52.77,52.8200,52.7300,52.730,424189.0,131.10,131.5000,130.8000,131.5000,32793.0
2017-10-20 04:00:00,25.13,25.1700,25.13,25.15,651942.0,52.605,52.614,52.5950,52.605,545736.0,...,52.65,52.6892,52.6301,52.640,286452.0,132.00,132.0800,131.7100,131.9500,32071.0
2017-10-23 04:00:00,25.15,25.1800,25.15,25.17,472528.0,52.625,52.640,52.6010,52.635,856312.0,...,52.68,52.7200,52.6700,52.720,262840.0,132.30,132.3000,131.2500,131.3300,29681.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,26.48,26.5200,26.47,26.52,4320797.0,54.620,54.640,54.6100,54.640,2304580.0,...,53.23,53.3100,53.1950,53.310,1291571.0,269.93,270.8400,266.5200,266.5200,161793.0
2021-10-01 04:00:00,26.54,26.5800,26.51,26.58,4107214.0,54.610,54.660,54.6041,54.650,2054957.0,...,53.28,53.3400,53.2300,53.330,1174288.0,267.62,270.0700,264.7936,269.3000,122595.0
2021-10-04 04:00:00,26.53,26.5799,26.50,26.55,2181656.0,54.610,54.630,54.6000,54.625,1516627.0,...,53.36,53.3700,53.3301,53.345,683339.0,268.26,268.2600,262.1500,263.9300,140997.0
2021-10-05 04:00:00,26.52,26.5300,26.46,26.48,4690593.0,54.600,54.600,54.5600,54.580,1570312.0,...,53.33,53.3350,53.2400,53.250,896557.0,264.99,268.9693,264.8400,267.4250,61595.0


In [ ]:
#Lower Risk portfolio (80% bonds and 20% stocks)

In [45]:
tickers_names =["TLT","VGLT","VOOG","VMBS","IWF","IGSB","GOVT","AGZ","AAPL","AMZN"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns = []
for column in columns:
    for ticker_name in tickers_names:
        if ticker_name in column:
            tickers_columns.append(column)
    

# ticker_columns
# new_df[tickers_columns]

In [43]:
lower_risk_portfolio =new_df[tickers_columns]
lower_risk_portfolio

,"('GOVT', 'open')","('GOVT', 'high')","('GOVT', 'low')","('GOVT', 'close')","('GOVT', 'volume')","('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')",...,"('AAPL', 'open')","('AAPL', 'high')","('AAPL', 'low')","('AAPL', 'close')","('AAPL', 'volume')","('AMZN', 'open')","('AMZN', 'high')","('AMZN', 'low')","('AMZN', 'close')","('AMZN', 'volume')"
2017-10-17 04:00:00,25.23,25.2600,25.21,25.25,675884.0,52.590,52.611,52.5800,52.599,621174.0,...,159.75,160.870,159.2300,160.47,16059637.0,1005.59,1011.4700,1004.380,1008.99,1487100.0
2017-10-18 04:00:00,25.20,25.2100,25.17,25.20,507468.0,52.575,52.615,52.5690,52.615,446712.0,...,160.42,160.710,159.6000,159.77,13668418.0,1009.27,1022.3100,996.550,996.99,1714126.0
2017-10-19 04:00:00,25.25,25.2650,25.22,25.23,689206.0,52.635,52.645,52.6050,52.630,529644.0,...,156.75,157.080,155.0200,155.98,32669974.0,990.33,991.0500,980.240,986.60,1963364.0
2017-10-20 04:00:00,25.13,25.1700,25.13,25.15,651942.0,52.605,52.614,52.5950,52.605,545736.0,...,156.61,157.750,155.9600,156.25,18427186.0,993.53,994.6200,982.000,982.91,1406057.0
2017-10-23 04:00:00,25.15,25.1800,25.15,25.17,472528.0,52.625,52.640,52.6010,52.635,856312.0,...,156.89,157.690,155.5000,156.20,16418890.0,986.73,986.7750,962.500,966.34,2341953.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,26.48,26.5200,26.47,26.52,4320797.0,54.620,54.640,54.6100,54.640,2304580.0,...,143.66,144.378,141.2800,141.32,76345831.0,3315.25,3327.8500,3273.990,3286.61,2349154.0
2021-10-01 04:00:00,26.54,26.5800,26.51,26.58,4107214.0,54.610,54.660,54.6041,54.650,2054957.0,...,141.83,142.920,139.1101,142.66,81585491.0,3289.01,3309.1746,3255.940,3283.17,2513022.0
2021-10-04 04:00:00,26.53,26.5799,26.50,26.55,2181656.0,54.610,54.630,54.6000,54.625,1516627.0,...,141.71,142.210,138.2700,139.19,86291346.0,3278.75,3279.9900,3176.250,3191.70,4034895.0
2021-10-05 04:00:00,26.52,26.5300,26.46,26.48,4690593.0,54.600,54.600,54.5600,54.580,1570312.0,...,139.49,142.240,139.3600,141.10,68998178.0,3204.12,3260.7300,3202.464,3220.99,2672848.0


In [50]:
tickers_names1 =["TLT","VGLT","VOOG","VMBS","AAPL","AMZN","BA","NVDA","BRK-A","GS"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns1 = []
for column in columns:
    for ticker_name in tickers_names1:
        if ticker_name in column:
            tickers_columns1.append(column)
    

# ticker_columns
# new_df[tickers_columns1]

In [49]:
#Low Medium Risk portfolio(60% bonds and 40% stocks)
low_medium_risk_portfolio=new_df[tickers_columns1]
low_medium_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')","('TLT', 'open')","('TLT', 'high')","('TLT', 'low')","('TLT', 'close')","('TLT', 'volume')",...,"('NVDA', 'open')","('NVDA', 'high')","('NVDA', 'low')","('NVDA', 'close')","('NVDA', 'volume')","('GS', 'open')","('GS', 'high')","('GS', 'low')","('GS', 'close')","('GS', 'volume')"
2017-10-17 04:00:00,52.590,52.611,52.5800,52.599,621174.0,125.39,126.090,125.390,125.92,3961713.0,...,197.09,198.9000,196.3500,197.72,10112393.0,244.44,244.89,235.190,236.09,5860461.0
2017-10-18 04:00:00,52.575,52.615,52.5690,52.615,446712.0,125.05,125.180,124.680,125.10,4706805.0,...,198.10,198.1400,193.3200,197.54,12459872.0,236.87,242.65,236.870,242.02,3322005.0
2017-10-19 04:00:00,52.635,52.645,52.6050,52.630,529644.0,125.74,126.010,125.290,125.29,7882061.0,...,194.44,198.0000,192.4501,197.79,12283460.0,240.30,240.99,238.100,240.03,1917186.0
2017-10-20 04:00:00,52.605,52.614,52.5950,52.605,545736.0,124.00,124.280,123.640,124.01,9720777.0,...,198.49,199.5900,196.5400,196.86,9450691.0,242.95,245.25,240.670,244.74,2310872.0
2017-10-23 04:00:00,52.625,52.640,52.6010,52.635,856312.0,124.16,124.490,124.020,124.20,4246265.0,...,197.70,198.8500,195.8000,196.62,9054036.0,244.39,245.33,241.381,242.16,1475595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,54.620,54.640,54.6100,54.640,2304580.0,144.10,144.515,143.650,144.34,19934128.0,...,207.64,210.6600,206.8800,207.14,19688760.0,388.00,388.02,376.620,377.81,2319747.0
2021-10-01 04:00:00,54.610,54.660,54.6041,54.650,2054957.0,144.81,145.400,144.200,145.34,22124468.0,...,207.51,208.5900,202.0300,207.42,21178667.0,379.44,382.72,375.845,380.00,2071258.0
2021-10-04 04:00:00,54.610,54.630,54.6000,54.625,1516627.0,144.62,145.460,144.100,145.00,19859275.0,...,205.04,205.4179,195.5500,197.36,31506939.0,380.21,386.87,372.820,373.98,2541908.0
2021-10-05 04:00:00,54.600,54.600,54.5600,54.580,1570312.0,144.60,144.700,143.461,143.59,19495879.0,...,199.48,206.4800,198.5400,204.50,24595681.0,377.00,388.22,375.500,385.85,2138397.0


In [52]:
tickers_names2 =["TLT","VGLT","VOOG","VMBS","AAPL","AMZN","BA","NVDA","BRK-A","GS"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns2 = []
for column in columns:
    for ticker_name in tickers_names2:
        if ticker_name in column:
            tickers_columns2.append(column)
    
# new_df[tickers_columns2]

In [ ]:
#Medium Risk Portfolio (60% stocks and 40% bonds)

In [53]:
medium_risk_portfolio= new_df[tickers_columns2]
medium_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'high')","('IGSB', 'low')","('IGSB', 'close')","('IGSB', 'volume')","('TLT', 'open')","('TLT', 'high')","('TLT', 'low')","('TLT', 'close')","('TLT', 'volume')",...,"('NVDA', 'open')","('NVDA', 'high')","('NVDA', 'low')","('NVDA', 'close')","('NVDA', 'volume')","('GS', 'open')","('GS', 'high')","('GS', 'low')","('GS', 'close')","('GS', 'volume')"
2017-10-17 04:00:00,52.590,52.611,52.5800,52.599,621174.0,125.39,126.090,125.390,125.92,3961713.0,...,197.09,198.9000,196.3500,197.72,10112393.0,244.44,244.89,235.190,236.09,5860461.0
2017-10-18 04:00:00,52.575,52.615,52.5690,52.615,446712.0,125.05,125.180,124.680,125.10,4706805.0,...,198.10,198.1400,193.3200,197.54,12459872.0,236.87,242.65,236.870,242.02,3322005.0
2017-10-19 04:00:00,52.635,52.645,52.6050,52.630,529644.0,125.74,126.010,125.290,125.29,7882061.0,...,194.44,198.0000,192.4501,197.79,12283460.0,240.30,240.99,238.100,240.03,1917186.0
2017-10-20 04:00:00,52.605,52.614,52.5950,52.605,545736.0,124.00,124.280,123.640,124.01,9720777.0,...,198.49,199.5900,196.5400,196.86,9450691.0,242.95,245.25,240.670,244.74,2310872.0
2017-10-23 04:00:00,52.625,52.640,52.6010,52.635,856312.0,124.16,124.490,124.020,124.20,4246265.0,...,197.70,198.8500,195.8000,196.62,9054036.0,244.39,245.33,241.381,242.16,1475595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,54.620,54.640,54.6100,54.640,2304580.0,144.10,144.515,143.650,144.34,19934128.0,...,207.64,210.6600,206.8800,207.14,19688760.0,388.00,388.02,376.620,377.81,2319747.0
2021-10-01 04:00:00,54.610,54.660,54.6041,54.650,2054957.0,144.81,145.400,144.200,145.34,22124468.0,...,207.51,208.5900,202.0300,207.42,21178667.0,379.44,382.72,375.845,380.00,2071258.0
2021-10-04 04:00:00,54.610,54.630,54.6000,54.625,1516627.0,144.62,145.460,144.100,145.00,19859275.0,...,205.04,205.4179,195.5500,197.36,31506939.0,380.21,386.87,372.820,373.98,2541908.0
2021-10-05 04:00:00,54.600,54.600,54.5600,54.580,1570312.0,144.60,144.700,143.461,143.59,19495879.0,...,199.48,206.4800,198.5400,204.50,24595681.0,377.00,388.22,375.500,385.85,2138397.0


In [60]:
tickers_names3 =["TLT","VGLT","VOOG","VMBS","AAPL","AMZN","BA","NVDA","BRK-A","GS"]
# new_df.rename(columns={"('GOVT', 'close')":"GOVT_close", "('GOVT', 'low')":"GOVT_low"})
columns = new_df.columns
columns
# ticker_columns = [x for x in columns if x in ticker_names]
tickers_columns3 = []
for column in columns:
    for ticker_name3 in tickers_names3:
        if ticker_name in column:
            tickers_columns3.append(column)
# new_df[tickers_columns3]

In [61]:
#Medium High Risk portfolio(70% stocks and 30% bonds)
medium_high_risk_portfolio = new_df[tickers_columns3]
medium_high_risk_portfolio

,"('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')","('IGSB', 'open')",...,"('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')","('GS', 'volume')"
2017-10-17 04:00:00,52.590,52.590,52.590,52.590,52.590,52.590,52.590,52.590,52.590,52.590,...,5860461.0,5860461.0,5860461.0,5860461.0,5860461.0,5860461.0,5860461.0,5860461.0,5860461.0,5860461.0
2017-10-18 04:00:00,52.575,52.575,52.575,52.575,52.575,52.575,52.575,52.575,52.575,52.575,...,3322005.0,3322005.0,3322005.0,3322005.0,3322005.0,3322005.0,3322005.0,3322005.0,3322005.0,3322005.0
2017-10-19 04:00:00,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,52.635,...,1917186.0,1917186.0,1917186.0,1917186.0,1917186.0,1917186.0,1917186.0,1917186.0,1917186.0,1917186.0
2017-10-20 04:00:00,52.605,52.605,52.605,52.605,52.605,52.605,52.605,52.605,52.605,52.605,...,2310872.0,2310872.0,2310872.0,2310872.0,2310872.0,2310872.0,2310872.0,2310872.0,2310872.0,2310872.0
2017-10-23 04:00:00,52.625,52.625,52.625,52.625,52.625,52.625,52.625,52.625,52.625,52.625,...,1475595.0,1475595.0,1475595.0,1475595.0,1475595.0,1475595.0,1475595.0,1475595.0,1475595.0,1475595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 04:00:00,54.620,54.620,54.620,54.620,54.620,54.620,54.620,54.620,54.620,54.620,...,2319747.0,2319747.0,2319747.0,2319747.0,2319747.0,2319747.0,2319747.0,2319747.0,2319747.0,2319747.0
2021-10-01 04:00:00,54.610,54.610,54.610,54.610,54.610,54.610,54.610,54.610,54.610,54.610,...,2071258.0,2071258.0,2071258.0,2071258.0,2071258.0,2071258.0,2071258.0,2071258.0,2071258.0,2071258.0
2021-10-04 04:00:00,54.610,54.610,54.610,54.610,54.610,54.610,54.610,54.610,54.610,54.610,...,2541908.0,2541908.0,2541908.0,2541908.0,2541908.0,2541908.0,2541908.0,2541908.0,2541908.0,2541908.0
2021-10-05 04:00:00,54.600,54.600,54.600,54.600,54.600,54.600,54.600,54.600,54.600,54.600,...,2138397.0,2138397.0,2138397.0,2138397.0,2138397.0,2138397.0,2138397.0,2138397.0,2138397.0,2138397.0


In [59]:
#High Risk portfolio(80% stocks and 20% cryptocurrency)

In [ ]:
high_risk_portfolio=  portfolio_prices_df[["AAPL","AMZN", "BA","NVDA","BRK.A","GS","COP","GILD","BTC","ETH"]]
high_risk_portfolio